In [1]:
%cd /ibex/user/slimhy/PADS/code/

from losses.rendering_loss import faces_scores, scale_for_renders
from util.misc import CUDAMesh

ROOT_DIR = "jupyter/data/rec_0/"

# Loading ground truth mesh
gt_mesh = CUDAMesh.load(ROOT_DIR + "orig_mesh.obj")
rec_mesh = CUDAMesh.load(ROOT_DIR + "rec_mesh.obj")

/ibex/user/slimhy/PADS/code


In [2]:
gt_mesh = scale_for_renders(gt_mesh)
rec_mesh = scale_for_renders(rec_mesh)

# Compute face-wise error scores
f_scores = faces_scores(gt_mesh, rec_mesh, render_resolution=512, num_samples=32)

In [3]:
import torch
from datasets.sampling import sample_surface_tpp

def sample_surface_weighted(mesh, num_samples, face_scores, squeeze_factor=0.5):
    # Sample faces
    weights = torch.pow(face_scores, squeeze_factor)
    weights_sum = torch.sum(weights)
    face_dist = torch.distributions.categorical.Categorical(
        probs=(weights / weights_sum).unsqueeze(0)
    )

    # Sample the mesh surface
    return sample_surface_tpp(mesh, num_samples, face_dist)

# Sample faces
pc = sample_surface_weighted(gt_mesh, 10**4, f_scores, squeeze_factor=0.8)

In [7]:
import numpy as np
import util.matplotlib_color_maps as mcm
import k3d
k3d.switch_to_text_protocol()

plot = k3d.plot()
plot += k3d.mesh(gt_mesh.vertices.detach().cpu().numpy(),
                 gt_mesh.faces.detach().cpu().numpy().astype(np.uint32),
                 # attribute=vert_scores.detach().cpu().numpy(),
                 triangles_attribute=f_scores.detach().cpu().numpy(),
                 color_map=mcm.plasma,
                 color_range=[0., 1.])

# Rotate y axis up
plot.camera_auto_fit = False
plot.camera = [0.5, 0.5, 1, 0, 0, 0, 0, 1, 0]

# Plot pointcloud with k3d
plot += k3d.points(pc.detach().cpu().numpy(), point_size=0.02, color=0xff0000)
plot.display()

Output()

In [5]:
rec_mesh.show()